### 实现目标
- 识别数字并存储数字
- 标识出数字的位置并画出图形  

### 方法
对模板以及银行卡进行轮廓检测，获取对应数字的轮廓矩形  
将银行卡中的矩形和模板一一比较，得到结果

### 问题
轮廓检测过程中会检测到许多不相关的轮廓，如何预处理剔除这些轮廓并保留自己需要的轮廓是主要问题  
通过长宽比例来过滤不需要的轮廓
- 预处理是图像处理中比较重要的部分

In [20]:
import cv2
import numpy as np

In [21]:
# 定义函数
# 排序函数
def sort_contours(cnts, methods = "left-to-right"):
    # 定义两个参数，决定排列的顺序
    reverse = False
    i = 0;
    # 条件判断，根据目标排列顺序改变参数
    if methods == "right-to-left" or methods == "bottom-to-top":
        reverse = True
    if methods == "top-to-bottom" or methods == "bottom-to-top":
        i = 1
    
    boundingbox = [cv2.boundingRect(c) for c in cnts] # 返回一个矩形，包括x,y,w,h
    (cnts, boundingbox) = zip(*sorted(zip(cnts, boundingbox), key=lambda b: b[1][i], reverse=reverse))
    
    return cnts, boundingbox # 返回排序完成的边缘和对应的方形结构

def my_resize(img, target_width):
    # img必须为2D图像
    h, w = img.shape
    ratio = h / w
    target_width = int(target_width)
    img = cv2.resize(img,(target_width, int(target_width*ratio)))
    ratio = w / target_width
    return img, ratio


def CV_show(name, img):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [22]:
# 轮廓导入
# 导入模板
template = cv2.imread("BankCardTemplae.png")
CV_show("BankCardTemplate",template)

# 将模板灰度化
template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY) # 灰度化

# 二值化
retval, template_binary = cv2.threshold(template_gray, 10, 255, cv2.THRESH_BINARY_INV) # 返回值为两个，需要两个变量来接收
# template_binary = cv2.threshold(template_gray, 10, 255, cv2.THRESH_BINARY_INV)[1] # 选择第二个变量返回
CV_show("BinaryTemplate", template_binary)

# 画出模板轮廓
ref_, contours, hirearchy = cv2.findContours(template_binary.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
draw_template = cv2.drawContours(template, contours, -1, (0,0,255), 2)
CV_show("draw_template", draw_template)

In [23]:
# 轮廓排序
ref_cnts = sort_contours(contours)[0] # 函数返回为元组，加中括号取第一个值
# 创建一个字典，用于存储所有的模板
dict_temlate = {}
# enumerate 将一个可遍历的数据对象转换为一个索引序列
for (i, c) in enumerate(ref_cnts):
    # 提取每一个数字的模板对应的区域
    (x,y,w,h) = cv2.boundingRect(c)
    roi = template_binary[y:y+h,x:x+w] # 从后往前为x,y,z等，即维度从后往前上升
    
    # 统一模板的大小
    roi = cv2.resize(roi, (40,64)) # resize又为(x,y)
    
    # 将获得的数字模板对应存储到字典中
    dict_temlate[i] = roi

In [24]:
# 对输入的银行卡进行预先处理，最重要的部分
# 初始化卷积核
rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
close_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10,10))

# 导入图像
BankCard = cv2.imread("BankCard1.png") # 彩色图像
BankCard_gray = cv2.cvtColor(BankCard, cv2.COLOR_BGR2GRAY) # 灰度图像
CV_show("gray", BankCard_gray)

# 归一化图像大小，更具有普适性
BankCard_gray, ratio = my_resize(BankCard_gray, 300)
CV_show("resize", BankCard_gray)

# 开运算
open = cv2.morphologyEx(BankCard_gray, cv2.MORPH_OPEN, rect_kernel)
CV_show("open", open)

# 礼帽操作 突出更明亮的区域
tophat = cv2.morphologyEx(BankCard_gray, cv2.MORPH_TOPHAT, rect_kernel)
CV_show("tophat",tophat)

# 闭运算
close = cv2.morphologyEx(tophat, cv2.MORPH_CLOSE, close_kernel)
CV_show("close", close)

# 二值化
binary = cv2.threshold(close, 127, 255, cv2.THRESH_BINARY)[1]
CV_show("binary", binary)

In [25]:
# 上述步骤将数字块状化，至此描绘轮廓，根据长宽比例找出数字块
ref, thresh_cnts, hirearchy = cv2.findContours(binary.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cur_img = BankCard_gray.copy()
cv2.drawContours(cur_img, thresh_cnts, -1, (0,0,255), 2)
CV_show("1", cur_img)

locs = [] # 存储目标字母块坐标信息

# 遍历轮廓，找到目标数字块
for (i, c) in enumerate(thresh_cnts):
    (x,y,w,h) = cv2.boundingRect(c)
    ar = w / float(h)
    # 判断边框的比例以及大小，筛选目标数字块
    if ar >2.5 and ar<4.0:
        if (w > 40 and w < 55) and (h > 10 and h < 20):
            locs.append((x,y,w,h))

locs = sorted(locs, key=lambda x:x[0]) # 对位置坐标重新排序
output = [] # 存储输出占据的空间

# locs 为处理后获得的数字块
for (i, (X,Y,W,H)) in enumerate(locs):
    roi = BankCard_gray[Y-5:Y+H+5, X-5:X+W+5]  # 划出感兴趣的区域 
    binary_roi = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1] # 将感兴趣区域二值化，勾勒轮廓
    cnts = cv2.findContours(binary_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[1]
    # CV_show("roi", binary_roi)
    sorted_cnts = sort_contours(cnts)[0]
    
    group_output = [] # 记录当前字母块得到的结果
    # 该循环为块内数字的确认与匹配
    for (j, d) in enumerate(sorted_cnts):
        x, y, w, h = cv2.boundingRect(d)
        target = binary_roi[y:y+h, x:x+w]
        target = cv2.resize(target, (40, 64))
        # CV_show("target", target)
        
        scores = [] # 记录匹配得分
        # 将当前数字与模板一一匹配，获取匹配得分
        for (num, numROI) in dict_temlate.items():
            result = cv2.matchTemplate(target, numROI, cv2.TM_CCORR_NORMED) # 相关系数，越大越匹配
            score = cv2.minMaxLoc(result)[1]
            scores.append(score)
        group_output.append(str(np.argmax(scores))) # 根据得分确认结果
        
    # 在原始图像中画出数字块与数字
    cv2.putText(BankCard, "".join(group_output), (int(X*ratio), int(Y*ratio)-15), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.65*ratio, (0,0,255), 2)
    output.extend(group_output)
    
print(output)
CV_show("BankCard", BankCard)

['4', '0', '0', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '1', '0']
